# Задание 
1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633690 sha256=4bc800b0cf09626123a411740e2631212fd11c216c16df1f707edc02d79f6e5f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)

In [5]:
# задаем датасет в формате surprise
dataset = pd.DataFrame({
    'uid': data.userId,
    'iid': data.title,
    'rating': data.rating
})

In [6]:
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(dataset, reader)

In [7]:
trainset, testset = train_test_split(data_surprise, test_size=.15, random_state=42)

In [8]:
# обучаем модель 
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [9]:
# делаем оценку качества на тесте
test_pred = algo.test(testset)

accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8676


0.8676343732520706

In [10]:
# оценка на кросс валидации
cros_val = cross_validate(algo, data_surprise, measures=['RMSE'], cv=5, verbose=True)
RMSE_mean = cros_val['test_rmse'].mean()
print(RMSE_mean)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8756  0.8725  0.8714  0.8638  0.8675  0.8702  0.0041  
Fit time          2.50    2.57    2.58    2.65    2.52    2.56    0.05    
Test time         0.31    0.19    0.33    0.17    0.17    0.23    0.07    
0.8701570928497553


In [11]:
# определяем пользователя и фильмы которые он посмотрел
user = 7.0
user_movies = data[data.userId == user].title.unique()

In [12]:
# формируем списки по предсказанию и наименовнию фильма
pred_rating = []
titles = []

for movie in data.title.unique():
    if movie in user_movies:
        continue
        
    pred_rating.append(algo.predict(uid=user, iid=movie).est)
    titles.append(movie)

In [13]:
# база фильмов которые пользователь не смотрел и прогнозный рейтинг фильму от пользователя
movies_not_watch = pd.DataFrame({'title': titles, 'pred_r': pred_rating})

In [14]:
# берем топ 10 фильмом из потенциального прогноза пользователя
most_interesting = movies_not_watch.sort_values(by='pred_r', ascending=False)[:10]

In [15]:
# формируем базу фильмов с средним рейтингом по фактическим оценкам пользователя
movies_with_mean_rating = pd.DataFrame(data.groupby('title').rating.mean()).reset_index()

In [16]:
# задаем функцию которая добавит средний рейтинг
def add_mean(row):
    return movies_with_mean_rating[movies_with_mean_rating.title == row].rating.sum()

In [17]:
# в понтенциальные топ 10 добавляем фактический средний рейтинг
most_interesting['mean_rating'] = most_interesting.title.apply(add_mean)

In [18]:
# создаем метрику которая взвешивает два подхода на 70/30 процентов
most_interesting['common_rating'] = most_interesting.pred_r*0.7 + most_interesting.mean_rating*0.3

In [19]:
#формируем топ 3 рекомендации, на основе финальной метрики
recommend_for_user = most_interesting.sort_values(by='common_rating', ascending=False)[:10][['title', 'common_rating']]

In [20]:
recommend_for_user

,title,common_rating
251,Pulp Fiction (1994),4.264742
271,"Shawshank Redemption, The (1994)",4.220441
2171,Fight Club (1999),4.149353
661,"Philadelphia Story, The (1940)",4.108190
451,Schindler's List (1993),4.074947
893,"Godfather: Part II, The (1974)",4.074257
895,"Grand Day Out with Wallace and Gromit, A (1989)",4.061228
3541,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",4.058270
4801,Eternal Sunshine of the Spotless Mind (2004),4.048138
930,"Great Escape, The (1963)",4.045197
